<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Enterprise Vector Store database functions
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<hr>

<p style = 'font-size:28px;font-family:Arial;color:#00233C'><b>Overview</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Teradata Vantage provides a suite of new in-database analytic capabilities for Vector storage, Management, Indexing, and Search, including</p>

<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Vector Datatype</b> based VARBYTE arrays</li>
    <li><b>Normalization</b> functions to improve search efficiencies</li>
    <li><b>Vector Indexing and Search</b> leveraging multiple algorithms</li>
    </ul>

<b style = 'font-size:16px;font-family:Arial;color:#00233C'>Vector Datatype</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Currently, the Vantage Database supports several different methods for generating <b>Vector Embeddings</b> including in-database Bring Your Own Model (BYOM) functions, in-platform GPU-accelerated open-source model inferencing, and API-supported embedding using Cloud-based Large Language Models.  However, not all of these methods support the native VECTOR datatype - and may return data as numeric or character-based columns.</p>

<p style = 'font-size:28px;font-family:Arial;color:#00233C'><b>Demonstration</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following demonstration uses some of the Consumer Financial Protection Board complaints data and embeddings, and will illustrate some of the in-database functions for constructing and analyzing vector data.</p>

<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Construct a VECTOR Datatype from FLOAT columns</li>
    <li>Normalize the vector data for efficient search</li>
    <li>Calculate Vector Distance between complaints and topics data</li>
    <li>Create an HSNW index and use it for highly-scalable distance analysis</li>
    </ol>

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Connect to the database</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Create a new named connection if necessary using the %addconnect magic</p> 

In [ ]:
%addconnect name=XXXX, host=XX.XX.XX

In [ ]:
%connect XXXX, user=XXXX

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Step 1 - Construct a Vector Datatype from float columns</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Many Vantage functions (teradatagenai classes, BYOM, OAF inferencing, and CSP-direct FastPath functions) generate Vector data as FLOAT columns.  Convert these to the new VECTOR datatype:</p> 
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Show original dataset.  This data is a set of vector embeddings generated with Open Analytics Framework using a Hugging Face open-source embedding model.  See that demonstration <a href = 'https://github.com/Teradata/lake-demos/tree/main/UseCases/GenAI/Complaints_Search'>here</a></li>
    <li>Run the native ClearScape Analytics Function "PACK" to construct a comma-separated array of FLOAT values</li>
    <li>Pass this data to the NEW VECTOR function</li>
    </ol>
<hr>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Show original data:</b></p>

In [65]:
SELECT TOP 2 * FROM demo_ofs.CFPB_Complaints;

,Date received,Product,Sub-product,Issue,Sub-issue,txt,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,id
1,2023-11-15,Credit card,General-purpose credit card or charge card,"Advertising and marketing, including promotional offers",Didn't receive advertised or promotional terms,"Request for Immediate Deletion of Unauthorized Inquiries from Consumer Report Dear Sir/Madam, I am writing to formally dispute the presence of certain unauthorized inquiries on my consumer report maintained by your agency, which violate my rights as a federal protected consumer under the provisions of 15 U.S. Code 1681c-2 and 15 U.S. Code 1681b. I request that you take immediate action to delete these items from my consumer report in accordance with the applicable laws and regulations. As a diligent consumer, I have thoroughly reviewed my consumer report and discovered the presence of inquiries that have been reported without my written consent, thereby contravening the provisions outlined in 15 U.S. Code 1681c-2. This violation is not only a breach of my privacy rights but also undermines the fundamental protections guaranteed to consumers under the U.S. Constitution. Moreover, in accordance with the "" saving to suitors '' clause as prescribed in sections 41 ( 3 ) and 371 ( 3 ) of title 28, U.S.C., I am asse",None,CAPITAL ONE FINANCIAL CORPORATION,NC,28110,None,Conse,Web,2023-11-15,Closed with non-monetary relief,Ye,**,7857635
2,2024-02-06,Checking or savings account,Checking account,Opening an account,Account opened without my consent or knowledge,A checking account was opened without my knowledge. I received a debt card in the mail XX/XX/XXXX telling me to activate the account. A few days letter I received another letter letting me know that the account was closed and to stop writing checks against the account. I did contact Citizens bank on XX/XX/XXXX around XXXX to let them know that I did not open the account. A representative put in a identity theft report for me.,None,"CITIZENS FINANCIAL GROUP, INC.",MI,481XX,None,Conse,Web,2024-02-06,Closed with explanation,Ye,**,8297421


In [66]:
SELECT TOP 2 * FROM demo_ofs.CFPB_Embeddings;

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383
1,7995584,0.016935547813773155,-0.03676247224211693,5.225228233030066e-05,-0.004347232636064291,-1.3892196875531226e-05,-0.05620070919394493,0.06395895034074783,-0.04171013832092285,-0.05411873012781143,-0.030588863417506218,0.04482881352305412,-0.010122044011950493,0.0007261849823407829,0.057559747248888016,-0.025537418201565742,0.04836396127939224,-0.01277296245098114,-0.06826107203960419,-0.060858599841594696,0.0630762055516243,0.04863440990447998,0.023036718368530273,0.002802856033667922,-0.033333901315927505,0.0035956597421318293,-0.007942725904285908,0.03508402034640312,-0.002601410262286663,-0.0925573781132698,-0.14888422191143036,-0.03870312497019768,-0.0706498920917511,-0.062703937292099,-0.05138754844665527,0.07859796285629272,0.03689837083220482,-0.08710163086652756,-0.006318407133221626,0.027627285569906235,0.009687943384051323,-0.025429818779230118,0.0414961539208889,-0.004856337793171406,0.004844540264457464,0.04187200963497162,0.024341002106666565,0.013187806122004986,0.02270432934165001,0.01227780431509018,0.017291387543082237,0.002914019161835313,0.02899782359600067,0.06198691576719284,0.07404793053865433,-0.021748056635260582,0.04933610185980797,0.0060989935882389545,0.038782864809036255,-0.035503119230270386,0.05611753836274147,-0.005625964608043432,-0.013889175839722157,-0.09438399225473404,0.016109317541122437,-0.04545862600207329,0.04313581809401512,0.010608769953250885,-0.018889466300606728,0.003740423358976841,0.08113187551498413,0.019115455448627472,-0.02715972438454628,-0.06146268546581268,0.03170076385140419,0.0032558676321059465,-0.012267662212252617,0.05869775637984276,0.02923036552965641,0.03511423617601395,0.1256742626428604,-0.02064884826540947,-0.02331252582371235,0.022787438705563545,-0.03841708227992058,-0.04411792755126953,-0.007160090375691652,0.04139092564582825,-0.015202837064862251,-0.011968361213803291,0.019567951560020447,0.04998449981212616,-0.0007488521514460444,0.05041259899735451,0.02463662624359131,-0.07445934414863586,-0.0012446806067600846,0.020398372784256935,0.04074062407016754,-0.08217578381299973,0.41216081380844116,0.006807119119912386,0.03796541690826416,-0.030163954943418503,-0.005089142359793186,-0.0028349219355732203,0.03902960196137428,0.00473461439833045,0.06561034172773361,0.03682667016983032,-0.05371808633208275,-0.024675626307725906,-0.03545799478888512,0.05856625363230705,-0.025529956445097923,0.012731538154184818,0.04482723027467728,-0.052393633872270584,-0.019723137840628624,-0.0023510486353188753,-0.02279377356171608,-0.05865446478128433,0.06573789566755295,0

In [67]:
SELECT TOP 2 * FROM demo_ofs.topics_of_interest;

,id,txt
1,2,Identity theft issues at Citibank
2,1,Fradulent activity with Debit Cards at Wells Fargo


In [68]:
SELECT TOP 2 * FROM demo_ofs.topics_embeddings;

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383
1,3,0.0025661527179181576,-0.07557964324951172,-0.032559074461460114,-0.03406253457069397,-0.06347081810235977,0.014943234622478485,0.021253304556012154,-0.033370401710271835,0.02218177728354931,-0.0689212903380394,-0.015974653884768486,0.017359504476189613,0.00019719969714060426,-0.0008609496289864182,0.06749783456325531,-0.03227228671312332,-0.044648319482803345,-0.02480117417871952,-0.05348867177963257,0.063388392329216,0.009803551249206066,-0.031176390126347542,0.02675558254122734,-0.015975769609212875,-0.03643132001161575,0.01798686571419239,-0.016748493537306786,-0.008153419941663742,-0.07843850553035736,-0.11344488710165024,-0.01603802852332592,-0.037371717393398285,-0.016314534470438957,0.00969875417649746,0.0523892417550087,-0.030027169734239578,-0.04508041590452194,0.043743252754211426,-0.023556368425488472,-0.006038457155227661,0.0338032990694046,0.019873401150107384,-0.0021433979272842407,-0.026125971227884293,0.007689967285841703,-0.0586981363594532,-0.010785559192299843,0.009344608522951603,-0.005452584009617567,0.027958620339632034,0.07725740224123001,0.0036774242762476206,0.004476992879062891,0.055257897824048996,-0.02317175641655922,0.024868551641702652,-0.01411520317196846,0.04107396677136421,-0.05303844437003136,0.049684979021549225,0.04531771317124367,0.01454611960798502,-0.11352081596851349,0.035966455936431885,-0.0015669078566133976,0.030496498569846153,0.011004198342561722,-0.042963333427906036,-0.007982735522091389,0.016875019297003746,-0.0180346816778183,-0.04347990080714226,-0.09286807477474213,0.018085969612002373,0.011979210190474987,0.03780669718980789,-0.020791975781321526,0.02312432788312435,0.009643558412790298,0.03772030398249626,0.004400772042572498,-0.04071925953030586,0.01959277130663395,0.044520970433950424,-0.08257842063903809,-0.001437770901247859,-0.043474338948726654,-0.005311567336320877,-0.015080262906849384,0.010061490349471569,-0.003466634079813957,0.04206645116209984,0.0849187970161438,-0.028225749731063843,-0.055021099746227264,-0.028819410130381584,0.015568836592137814,0.027609389275312424,-0.07057931274175644,0.3119887411594391,0.024235783144831657,-0.008769078180193901,-0.013838057406246662,-0.04740266874432564,0.051366277039051056,0.0056484476663172245,0.010318608954548836,0.014861696399748325,-0.0037530299741774797,-0.022364797070622444,-0.00035473896423354745,-0.04261234775185585,0.03757600858807564,-0.06579847633838654,-0.024893052875995636,0.10623007267713547,-0.0278522539883852,-0.03315160050988197,0.03316415846347809,0.0026331632398068905,0.011561221443116665,0.0007899

<hr>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Pass the output of PACK to the datatype constructor</b></p>

In [22]:
SELECT NEW VECTOR(packed_data) Vector_Data, id

FROM (
    SELECT * FROM PACK (
    ON (SELECT TOP 2 * FROM demo_ofs.CFPB_Embeddings)
    USING
        OutputColumn('packed_data')
        TargetColumns('[1:384]')
        IncludeColumnName('False')
        Accumulate('id')
) AS dt) d;

Vector_Data id 1 fbffffdf39cfa0bff3ffff5f19ce85bf010000a0f43ab2bfffffffdf68a4a83f0400006002167dbffdffffff6f07b6bffcffff5ff51b893ff9ffff7f08c5a2bff5ffff1f1a7a91bfffffff9fff34b4bff6ffff5f7454943f02000080e440a03f070000c0c20ba0bf0d000060c228943f0300002063e337bf030000c04aa8a43f040000003b6ca33f03000080e572aebf020000a031b8a0bfffffffbf1341a03f020000a08fe9a03f030000009c3299bf010000a08ac488bfffffff1f69d483bf030000c01138b13fffffffbf95ed973fefffff3f7ce58dbf0d000000eb2c913ffdffff1feedcb1bffbffff1f7fe7c1bf020000c07f38693ffbffff7f8b69a4bf010000c000e9b03ff2ffff9fdb06963f020000e0fc64b13f090000c07694933ffeffffbfe613aebfefffff5f2f1c863f06000080c0dd7dbf090000608bd7993ff2ffff3f49d096bf040000200238a63ff9ffff5f9add9dbffbffffbfc30075bf010000005e63a53ff8ffffff4c5491bf01000020971da73ff4ffff5fbc5290bff5ffff7f2693973ffcffffff19bf9e3f080000807213963f030000e07fd9a2bf020000e0986d833f05000000c6beaf3f000000800e4a81bf030000e0d404a23ffbffff9fd4a1af3ff4ffffdfa37886bf020000e0af6fb13ffeffff1f5cf7a83f060000403744653f000000e0f74bb0bffaffff5ff891c0bf04000040bd06a13f03000040ea427c3f040000c0dd26a43f0200006001d6753fffffffbf0f408f3fefffffdf1696883f170000e0452ebc3fffffffff58ea833fffffff3f817ea0bf030000a00132b1bffdffff1f02179d3f040000208893623f010000c0453574bf05000080ee06ac3ff4ffff9f015098bffaffffbf761aa53fffffff7f915bb33ff2ffff1f39169bbf25000000ec4dbcbf0e000060d1de5d3ffcffff3f94cd9fbf18000060692359bffcffffdffd57a4bf0c0000401b2d9cbf090000e000ad653f000000a02405b53f060000e03945693f000000a059d0a13ffbffffff2111a7bf03000000c5cdb23ffbffffbf2522a9bffcffff5f9492b0bffcffffdfc88aaebf020000e0bf1d9b3fffffff9fce61a13f010000202543b4bf010000602fa8dc3f03000060a4babb3ff2ffffbfdff99f3ffeffffff7c63913ff2ffffffdf39913ffaffffbfb700a43f010000603db5a2bff8ffffdf072595bf020000c0765eb23fffffff1f5fc0723f0a000060f5ed9abf03000060a4cea0bfe9ffff7f5c12863f01000060e83cb33f0e0000c00ff6883f020000400573b1bf050000201b04aa3ffbffff1fb8dca1bf0200000062fb763f0d0000a099be993ff3ffff1fe0d3983ffbffff7fd004abbff9ffffbf6dd0ac3ffbffffbfc557783fe7ffff5f0a5c8ebffdffff1f3367aabf06000020aa01a0bf020000a0e80792bf030000a042a3b63f080000c0335e61bf02000080662d933ffeffffbf2796a83f03000060e512b5bffcffffbfac16b0bf05000060383074bfffffff3f7deba23f05000020483b943ff5ffff5fcb50613ffaffffbfcfd2a93ffdffffbf5bbbb63f03000060e9d79cbf03000040a0337fbf040000e05bdea4bfebffff9f20d58b3ffaffffdf5d32943f02000080f32272bfffffff3fef1f973f02000040282c7abff8ffff9f07a5943ffdffff7fcc9e7dbff6ffff3fb0b460bf0c0000009d2d8bbf15000080699f87bff3ffffdf8a4a963f050000a0a524adbff3ffff7f25eb8b3f010000206a8aa23f060000a09534a93ff9ffff1f39359d3ff5ffff5f949891bf080000c021a0613ffeffff3f897ea03f00000040cb2ea33ffdffff1ff6b9b0bf01000000a79db23ff1ffffffd663bb3f050000807075a5bf010000405dfe72bf090000c0de0856bff6ffff7f78fb9fbf050000c0b935a03ffdffff9fdd4595bff4ffff5ff9c994bff9ffffff7929a83ffdffffdf12a3603f050000c0cdd6a83ffcffffdf8badaebf060000007542a7bff6ffff7ffaff8dbf04000060d19184bff5ffff5fd54c9e3f05000080a901acbffbffff5f0d428d3f040000e09cb098bf04000040b17f9fbf040000603551723f0c000000e2da96bffbffffffe43d93bf050000001f71a4bff5ffffbf7fad8d3f020000208fd5b4bffcffff9f32e5a8bffeffffffeb4881bfffffffbfd6a4b4bf050000200749ad3ff3ffff1f25b596bf00000040d33895bf190000605bf18cbf020000a0f95c833ff8ffff7f03229cbffeffff5f2f3293bf020000e03520b73f05000000179591bffeffffbf84f9a13f020000a0f013a83f15000080c6ce8b3f020000207a9b7e3f020000e0f081923ffbffffdfd81aa2bff8ffffff170f923f120000401a4086bf03000000e5d2a63ffdffffbf5ace753f010000002dfda63f00000000f212783f080000a04d509a3fffffff9f484ca03ffeffffbf2bff833f01000080a5c77c3ffcffffdf3cb4623f03000080f9d1a7bfffffff7f5dcc7f3f070000200cac9e3ff8ffff5fb67c9d3fffffffbf9b30d4bff8ffffdf95289dbffeffffdf8059b2bffdffff3fb6e682bf030000e01934b1bf060000e06c86abbffaffff3f2955a53ffeffff9f8078b0bf020000405698b7bf040000c0f1d7b53ffbffffdfb17c7d3f020000c09674703ffcffffdf83d3a73f0d000020ffd78ebf04000000a50c9b3ffdffffbfa4ed953f030000006310793ffeffff7f23f9b33f050000200fd596bf050000208973963f000000809e03b7bffeffff1f0e4980bf010000e0783a70bf010000406fd47a3ffaffffff8a27a43ff4ffff7f970b983

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Step 2 - Normalize the vector values</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Vector normalization is the process of scaling a vector to have a magnitude (length) of 1, while preserving its direction. This resulting vector is called a unit vector. It's essentially dividing each component of the vector by its length.  This makes some calculations much more efficient, including some of the search and indexing operations.  The ClearScape Analytics function <b>TD_VectorNormalize</b> will perform this operation at scale on our VECTOR datatype.</p>

<hr>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Review the queries for both tables</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Complaints Embeddings and Topics Embeddings</p>

In [13]:
SELECT TOP 10 id, NEW VECTOR(packed_data) Vector_Data

FROM (
    SELECT * FROM PACK (
    ON (SELECT * FROM demo_ofs.CFPB_Embeddings)
    USING
        OutputColumn('packed_data')
        TargetColumns('[1:384]')
        IncludeColumnName('False')
        Accumulate('id')
) AS dt) d;

id Vector_Data 1 7838890 010000c0d7ffb6bff9ffffdfa686a5bff7ffff1fff22613ffeffffdfb930aa3ffdffff5f485d713f0c000040dae888bf020000c0ecbaa33ff8ffff9f75c89d3f1c0000a03c358a3ffbffff1f73df8abf060000e0587b873ffaffff3fd439953f010000c003705dbffbffffbfec7fa13ffaffff9fb3a9a2bf03000060ed9ca53ffbffff3f01f9a7bf230000e0a179bdbff6ffffbf166b97bf030000e0dc72a43ffeffffff4bfba23f09000080a9ea643f0700000000939ebfffffff1ffbeaa93ff7ffffdfb995873f060000a00b7c843ff4ffffffd8b3953ffbffffffe030943f020000208b84b1bff6ffff5ffa76c2bffbffffbf59be763ff6ffff7f12ee93bf030000c046a64e3ffeffff3fcad6a5bf030000809927813f0b0000a0bf4e66bf0900004058e43d3f04000000808393bf08000000d84c9a3f030000a0cc60b33ffbffff9f931f6f3f05000060a57cae3f040000800230a43ffdffffbf9431adbffbffffdfbe6c7dbff4ffffff66fb8fbffbffffbf62e57f3ff4ffff3f4a70873f040000c06342af3f020000203e73b33ffeffff7f2f9880bf000000c081ceaabf180000e09fea8d3ff4ffffbf93c69a3fffffff5f94716c3ffbffffbf217dab3f020000e09fbd943f060000a085be9a3f05000040c9487bbf07000060cfc8873f010000401f6e833f03000020ed8c793f0b0000c077b7c2bffeffff5f2b3db63f050000209534afbff3ffffff8a649f3ffbffffbf40336b3f1c000060c62385bf1b0000a0a5cf8a3f050000c0fea2a63f030000e0bc22a1bfecffffdf3de88dbf00000060a648b3bf020000203ae89c3ffeffff5f2371a73ffaffffdfd383acbf020000e07acca23f01000080b7fdb43ffbffff3f3ee0a53ffbffff7f52efa83ff5ffff9f77499fbf020000207bdbb3bf020000804a98a2bf09000060797962bffdffff5f9088b23f00000040d3b7b0bf010000a0b362933f090000003e2e613f030000e0c4d3b13ffaffff9f3193993ffcffffbfc403a83fffffff7f50d4b0bf0c0000c0901e8cbffbffffbfdce370bffeffff7f2747b6bffdffffffee70753fffffffffc07e71bf09000040157c9a3f07000080ef729dbff8ffffdfe0e9db3f01000080bb82a23f0000002051b4b03ffeffffbf68c8b4bf02000020c89dac3ff2ffff3f76139ebf020000405262a3bfedffff5f3509533f03000000f2287ebfeeffff1ffe6b873f0e000000048799bf01000020a775a1bfffffffff9d6fb13fffffffffad3dac3f0800002039f391bffdffff9f4363a7bff6ffff5fc959be3ffdffffbf63f7b13ffaffff9fecaba53ffcffffdf7f5688bf08000000f0069abf1f000000c04b8fbf030000609726703f030000a0b73346bf020000e0f05bad3f030000c0b407b5bf02000000a85ab3bf02000080528ca73ffdffff3f37ddb73ffdffffbfa06aacbf000000e0ba08ad3f070000409251a63f06000080581da7bf050000e0fe30a0bfffffffff70a2a2bff1ffff3fe35e873ffbffff9f0d2eac3f0d000080ce39993f0200000005618b3ffcffff7fb479a23f03000040b0db93bf060000c0d68067bfffffffbf9906b4bf10000020d89584bf040000a06b5c9d3f050000a0b1fd773ff9ffff9fe7d4ae3f0c0000a02f789cbff8ffff9f504f98bffaffff1f25e4a1bf0d000040910293bff9ffff5f55b890bf020000e0587a6c3f0400000043d898bffdffff9f9c8daabff4ffff7f4e159bbffbffff5fd8489e3f000000e009c2743ffbffffdf9ef9793ffcffffdf77c092bf0000004049a66b3f040000c06bccab3f070000a0f7fa90bf0e0000e08d979bbf040000204bbbb03f00000060086ca13fffffffbfd6f5b4bffaffffdffc457ebf020000405f486c3ff4ffff9fd8a98f3fffffffdfd2ea90bf040000c0e911afbffcffff1f0efaa03f05000080835e78bffbffff7f9b45adbfe9ffffdfde86bb3f040000603529753f000000a06192a13f03000000124c7c3ffdffff5f740382bf010000000c8682bf0b0000a016d29fbf0b0000200ebc903ff8ffffbfce3491bff6ffffbf577f923f000000407e37a53f060000403c46a9bffdffff3f700084bfeaffffbfc2d784bffaffff1f2514a53ffcffff9f6beba4bffcffff1fe533a5bf01000080f438acbf010000e04511b6bf03000020161fb23ff1ffff7f7587893f0000004004c1b3bf040000204c68313f02000040c31f7a3f01000020b3f8b03f13000060f6d88dbf010000a04d1eb73f11000000acc85e3f03000040266598bf01000040ed237b3ffcffff5f29324a3f0200002083fab03ffdffff3fc5cbad3ffdffff5fbd8eb2bff4ffffdf5244903f060000607522913f030000203b1d673f090000a0b7e6983f050000a0507a73bff6ffff7fb566943f00000060249f983f020000805a62b83f0000000041e6ad3f0200008010fb803f1500002020778bbf1a0000c00eec88bffeffffdf085a6bbffcffff9f88a17ebffdffff7fbd38b83f02000060e0cad3bf010000c05d2f90bff6ffff7fc9cf9dbff7ffff1f17599cbffbffff9f5af1923f040000a0658c78bff3ffff9fa7299a3ff4ffffbf5e1194bffeffff9ffad482bf07000060dfb0a43f0400002084a8ab3f160000a0967c55bff5ffff9ff5f499bf0700008096dc8bbffcffff1f3df07dbf030000201a9eabbff7ffffffb9a3923ff8ffffbf9dc09f3f1a0000e00ebe8f3f030000008488b9bf06000000efb4adbf030000002f4c84bf080000c010ff91bffaffff1f627dacbf070000a0be24993ffaffff3

In [14]:
SELECT TOP 10 id, NEW VECTOR(packed_data) Vector_Data

FROM (
    SELECT * FROM PACK (
    ON (SELECT * FROM demo_ofs.topics_embeddings)
    USING
        OutputColumn('packed_data')
        TargetColumns('[1:384]')
        IncludeColumnName('False')
        Accumulate('id')
) AS dt) d;

id Vector_Data 1 6 0b0000c0ac7d97bfffffffdfe282b1bf03000000a2d972bff7ffff3f1b4499bf1200006026558ebf010000c082fbb4bf010000a0c344b83f06000020303190bfffffff9f44f9aa3f0c0000e043bd97bf0d0000c0d6ab983f010000c0519bb0bf01000000772280bffcffff7f5268763fffffffdfd61eb03ff8ffff1f3adb693ff5ffffdf4ae6983fffffffff2f30b7bf050000603d2e613ffcffffbf6f69b73fecffffdf8ab387bfffffff7f38129fbf02000060c8aab2bffbffffdf3155afbfffffff7ff7a3a03febffff7f4985893ffaffff1f1d729d3f01000080f0e6b0bfffffff3fef76b0bf020000c07253b3bf070000004d4ead3f170000e0f8788f3f010000607a74afbf05000000487da3bffeffff7f9515b43f0000002049dc9a3ffaffff3fd8a2a7bf0f0000407a00853fffffffbf1baf953f020000403ee7a3bf06000080d2fb903f03000000d548a33fffffff1fe363b5bfffffffbfd2f4b5bf04000080a077a43f04000020c54e943f020000a0a32bb23f06000080f4b5753ff3ffff5f1996c73ffbffff3f52fe4dbfecffff5f1d288cbf0b0000a0b0ec9f3fffffffdfe631a53f060000407981a13f02000040092a9a3f080000e0cbef913f01000040032b7e3ffcffff9f2a1ba43f0d000080858792bffdffff1f7c51a1bff9ffffbfe39d893ffeffffff573ea3bffeffff3fde47b9bf030000008ca29f3ffbffff5fcc89a73ff8ffff5f6568973f040000e0737caf3f060000e091a5a63fffffffdf8d23a93f0e0000e0408c87bffdffffdfdd7da13f020000601d09afbffdffffbf2a0da5bffeffffffc9bcab3ff6ffffdf193967bffcffff1f0e5ca2bf070000802c4ca33ffeffff9f9d27a3bf000000c038c2a03f0c000060b9d29c3f05000020d61aabbffdffff1f3b75b2bff3ffff9f3d9797bf050000000b54a03fffffff7fbed3a6bffbffff7ff8e1793f020000a0f0757ebffeffff5f693184bf02000060ff03793f040000202fc596bf040000e02a807e3f05000080ad92af3ffeffff9f7c7cb53f10000060d460853ff9ffff5fdb45993ff2ffffffaea9953f020000e039d49d3ff4ffffdf5a0a9abf0b00004043539ebfffffff9f8b2fd43f030000e05e27b03ffdffff5f339eb23f0600002065bca2bf0d000020f7ad8ebf05000000c6dc943ffdfffffffdb9183f030000a0f00079bf020000a05556843f04000020faf5a1bffeffffffddee47bffdffffbfef79803ffdffffbf5f22af3f080000205ab0c13ffdffffff6e5f58bf0700004049159fbf040000a0e17fa33ffaffff7ff0b4a6bffdffffff6f34a0bf000000409e49b9bffcffffbf73c99c3ffeffff1f8eaa79bff5ffff3f880f993f060000a0235296bffeffff9fede0793fffffff1faad2aabf03000080a83da5bffaffffbfb7aca8bfffffff9fff8db23ffeffff1f4cad943f0000006055eb993ffdffff1ffe7cb03ffdffff1f434ba7bff9ffff7f5d42a9bf000000207accafbfffffffbfedd5833f010000a01086a2bf000000a00b787ebf030000803efdac3ffdffff5f471c9d3f0c0000406c7199bfffffffbfde3b90bf000000a020d2b5bf010000a0b65681bf040000e03639a3bfefffff5ff63d8b3ffeffff5f065b84bffdffff3f7ce3a2bfffffffff54bd933f16000080495f853f070000205943afbfffffff9f5c8996bff8ffffbffbb79dbfffffff9f5cc179bf000000805dc9acbf040000c032a87dbf060000e06ce8a73f00000020927ca93f000000a0aa35b23febffffbfcf2a8a3f0e0000a071d0983f030000e0c093a6bffdffff9f0630abbf030000407357b3bffaffff7fb35590bf020000e0bf35a53f010000c00473a6bfffffffdf4c17abbf1d000000fc0a87bf000000c0380ea3bf030000e05abc78bf0d0000e01b1f90bf00000080370480bf020000e0a3abb4bffaffff3f669a973f190000409c6bbc3f070000e081c3abbff2ffffdfb2449d3f07000040cffa95bf040000c0678d983f030000e07b30a03ffaffffff926997bffaffffbf82d597bfffffffdf8dc9adbffbffff3f7187733f060000a0991b9a3f0a000080da3660bf100000609b1a88bffcffff5f5745b1bf01000040a7c8b13ffdffff9fd10aa0bf010000002d3790bf060000402b6169bf06000000f325c0bffaffff7f39279ebf00000080dbe6a6bfffffffffe6dbadbfecffff1f7665543ffdffffbfae8d813ffbffffbf1dac9c3f060000400374773f0100000062c2c63f07000060db29a93ff4ffff1f63ac93bff9ffffdf286fa83fffffff1fdb90a2bfffffffbf1313af3ffbffffbf6ac9a5bf030000e00987b9bff5ffff3f53d690bf1d0000801c03853ffcffffbfb0faab3ffeffffbf14fb7c3f000000e0b2d7b23ffcffff3f4fd09b3f1e0000c04d6b86bf0d0000800f369cbf050000c08c7ead3f0000000049aca23ff4ffff5f01789c3f0d0000e0a97a91bf080000e09255973f00000020865395bff6ffff5f780993bf030000a04416d2bf030000405721a33ffaffff9f45fc9dbff9ffff3fcbab9fbf000000404689123f000000408a5695bfffffff7fceab933ff7ffffbfb54290bf160000a0d79c8fbf0000008059b6a13f020000c01262823f030000e0a5f061bf0c000020e290913f08000040627b943f0a0000a0c078853f000000e0342374bf000000203d2cb5bf000000802cfbb13f000000209b156abff3ffff9f29e291bfffffffbf0b9ba1bf090000a09f3090bf010000c01891b8bffeffff5fa5b391bf060000407502a43ffeffff3fda88a

<hr>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Execute the VectorNormalize function to create new tables</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Create new volatile tables passing all the functions in a single expression.</p>

In [16]:
SELECT TOP 10 * FROM TD_Vectornormalize(
   ON (SELECT id, NEW VECTOR(packed_data) Vector_Data
        FROM (
            SELECT * FROM PACK (
                ON (SELECT * FROM demo_ofs.topics_embeddings) --original FLOAT-based embeddings table
            USING
                OutputColumn('packed_data')
                TargetColumns('[1:384]')
                IncludeColumnName('False')
                Accumulate('id')
        ) AS p) d) AS InputTable
USING
    IDColumns('id')
    TargetColumns('Vector_Data')
    Approach('UNITVECTOR')
    EmbeddingSize(384)
) AS dt;

id Vector_Data 1 6 47867dc2ac7d97bfe03ddbe1e282b1bfc193ff01a2d972bf1ab3ad421b4499bfe933376326558ebf6e7239c282fbb4bf5ca192a2c344b83f0c71b721303190bf580bdca244f9aa3f124484e243bd97bfc58e9dc2d6ab983f59b1c2c1519bb0bf74e1b501772280bf191f60825268763f0e7fb5e1d61eb03f66b8bd223adb693f10c1a3e24ae6983f4a4f75023030b7bfaa44d2613d2e613ff8607bc26f69b73f153c83e28ab387bf103f4b8338129fbf479cfa61c8aab2bfac5852e33155afbf069cc381f7a3a03f029cb4824985893f3e221f231d729d3f9db5ca81f0e6b0bfe3d5be41ef76b0bfb97d0cc27253b3bf5b561b034d4ead3fc02356e3f8788f3fb2a955637a74afbf0ced1002487da3bf641221829515b43fcaf8d82249dc9a3ff9768142d8a2a7bf47f939427a00853f8e7c4cc21baf953fc1281c423ee7a3bf60eccc81d2fb903f9c5d0b02d548a33f1d834422e363b5bf8fe053c2d2f4b5bf39772b82a077a43f66222722c54e943fb421eda1a32bb23f65364d82f4b5753f0a1d80621996c73f5dff2d4352fe4dbf4826fc621d288cbf306862a3b0ec9f3f8d363fe2e631a53f9117db417981a13f4013c642092a9a3fa5c9e6e1cbef913f46bc3243032b7e3fe4a921a22a1ba43f5cdff681858792bf2801d6217c51a1bfc137b7c2e39d893ff2400a02583ea3bf3819ae42de47b9bff98b5a038ca29f3f3dcf7e62cc89a73fb7447b626568973f228256e3737caf3f589d66e291a5a63fb13faae28d23a93ff1117fe2408c87bfa4b5dae1dd7da13ff3474a631d09afbf97513bc22a0da5bf9fc5f002cabcab3f294176e2193967bfa743f2210e5ca2bf52b80b822c4ca33f21d807a29d27a3bf1fd1c6c138c2a03f9b3c0e63b9d29c3f639adf22d61aabbfe8eef4213b75b2bf053c80a23d9797bffb22bb010b54a03f73826b82bed3a6bf6d6fbe82f8e1793fbead3aa3f0757ebf9c052462693184bf48e7a662ff03793f52f769222fc596bf51c33be32a807e3f4fdd5883ad92af3fb91e47a27c7cb53f33304462d460853f9ee2ad62db45993f47e94b02afa9953ff68829e339d49d3f65b7c2e25a0a9abfab00374343539ebff8d223a28b2fd43f9a66b6e15e27b03fca46f961339eb23f477afc2165bca2bf459e4023f7ad8ebf42303602c6dc943fd10e9f02feb9183f4e94a6a2f00079bfafef27a25556843f5e71e721faf5a1bf2d868902deee47bf5927bfc1ef79803f72f54cc35f22af3fd10fe0215ab0c13f227595026f5f58bf4a924b4349159fbf9c3311a2e17fa33f6b3e6882f0b4a6bf34c9b7017034a0bfb948ae429e49b9bfe9400dc373c99c3f818fb8228eaa79bf4620a842880f993f1bc55da2235296bf2553bea2ede0793fadf3d722aad2aabfa0754082a83da5bf8da69dc2b7aca8bf168ff7a1ff8db23fc52731224cad943f8b6dbf6255eb993f4a7abf21fe7cb03f112e7822434ba7bfdb83ad825d42a9bfeafd5e237accafbfd8521ac2edd5833fc8b7f6a11086a2bfe0e63aa30b787ebf88be12833efdac3fc3081663471c9d3f0981b2426c7199bfed92b8c1de3b90bff23250a220d2b5bf118fd6a1b65681bfc3b509e23639a3bf9753e362f63d8b3ff66e2862065b84bf1f9f00427ce3a2bf4fb7170255bd933f5c064482495f853f607450235943afbfc69f63a25c8996bf708a26c3fbb79dbf77fabaa25cc179bf913e0d835dc9acbf17de24c332a87dbf61d788e26ce8a73f89afb322927ca93fd531eea1aa35b23f3428c6c2cf2a8a3f3470a1a271d0983fcfb964e2c093a6bf6bd9e1a20630abbf56ea0c427357b3bff04fbb81b35590bffc9e3fe2bf35a53f684161c20473a6bf653adfe24c17abbfbe5d7102fc0a87bff62605c2380ea3bff74e9fe25abc78bf6c86b5e11b1f90bf8bacb281370480bfcffa30e2a3abb4bfc6918042669a973f3c4e03439c6bbc3ffc7bf1e281c3abbfaf511ae3b2449d3f0e835442cffa95bfcb549ac2678d983fed5db7e17b30a03fb2647b02936997bf02d686c282d597bf536728e38dc9adbfc60012427187733f7c8bc4a2991b9a3fd20ab881da3660bf4e298e629b1a88bf96b7d4615745b1bf4ea3e241a7c8b13fb55fb3a1d10aa0bf8813b8012d3790bfe5c7b0422b6169bf0940b601f325c0bf7155328339279ebf2c896d82dbe6a6bf47592a03e7dbadbf218a29227665543fdf62dcc1ae8d813fc1240ac31dac9c3f09807c420374773f49ab690262c2c63fc6eaaa62db29a93f66eb152263ac93bfe91f97e2286fa83fa2dcf721db90a2bf4f564bc31313af3f88464fc26ac9a5bfa2cbb4e20987b9bfacf2c84153d690bfd0403a821c03853f9155f7c2b0faab3fce8312c314fb7c3f415fffe1b2d7b23f62d7f2424fd09b3f257060c24d6b86bfe4a0fd820f369cbfca7320c38c7ead3f0bc5fa0149aca23f7b9e046301789c3fc65edae1a97a91bff44579e29255973f08c74222865395bff9a50462780993bfb7ddeaa14416d2bfda2d07425721a33fbec72da345fc9dbfe5865b43cbab9fbfe10ef7414689123fe31843428a5695bfafdb1582ceab933f854cb9c1b54290bf3bf159a3d79c8fbf8eb2e08159b6a13f00e7f2c11262823fbce0e6e1a5f061bfccb9dc21e290913f32dd2b42627b943f6cb946a2c078853f1b8422e2342374bfdd9c3e223d2cb5bf63fee7812cfbb13fc9e8c3229b156abf8d57e5a129e291bf8dcdddc10b9ba1bfbe61b7a19f3090bff7b89ac21891b8bf2c69e061a5b391bf5c0b1f427502a43f36d99942da88a

In [ ]:
DROP TABLE topics_embeddings_Normalized 

In [34]:
CREATE VOLATILE TABLE topics_embeddings_Normalized AS (
    SELECT TOP 10 * FROM TD_Vectornormalize(
       ON (SELECT id, NEW VECTOR(packed_data) Vector_Data
            FROM (
                SELECT * FROM PACK (
                    ON (SELECT * FROM demo_ofs.topics_embeddings) --original FLOAT-based embeddings table
                USING
                    OutputColumn('packed_data')
                    TargetColumns('[1:384]')
                    IncludeColumnName('False')
                    Accumulate('id')
            ) AS p) d) AS InputTable
    USING
        IDColumns('id')
        TargetColumns('Vector_Data')
        Approach('UNITVECTOR')
        EmbeddingSize(384)
    ) AS dt) WITH DATA
ON COMMIT PRESERVE ROWS;

ERROR: Unable to run SQL: Unable to run SQL query: Database reported error:3803:Table 'topics_embeddings_Normalized' already exists.

In [22]:
DROP TABLE CFPB_embeddings_Normalized 

Success: 0 rows affected

In [24]:
CREATE VOLATILE TABLE CFPB_embeddings_Normalized AS (
    SELECT TOP 10 * FROM TD_Vectornormalize(
       ON (SELECT id, NEW VECTOR(packed_data) Vector_Data
            FROM (
                SELECT * FROM PACK (
                    ON (SELECT * FROM demo_ofs.CFPB_embeddings) --original FLOAT-based embeddings table
                USING
                    OutputColumn('packed_data')
                    TargetColumns('[1:384]')
                    IncludeColumnName('False')
                    Accumulate('id')
            ) AS p) d) AS InputTable
    USING
        IDColumns('id')
        TargetColumns('Vector_Data')
        Approach('UNITVECTOR')
        EmbeddingSize(384)
    ) AS dt) WITH DATA
ON COMMIT PRESERVE ROWS;

Success: 0 rows affected

In [25]:
SELECT TOP 10 * FROM CFPB_embeddings_Normalized;

id Vector_Data 1 7854870 7fa8cba5c85059bf77043dc5b6e196bf9ff3ddc6edfeadbf5c12b36563e5783f29167a8578eca73f97e9e886ce2eaebfb4b81705cd3ea63f7e1cce8443fda43feaeeee65deea89bf66d0948674bf9cbf5497e564d36375bff755214745257fbf5d99fbc66e809ebfa18a5b44cf08733f505cb0c58bd9a8bfcf4f9b059a7da83f8fbd78a45a88933f9ad2f1a3043b81bf58e47686c03c5c3f5f117b448592b33fe70c8b8455d8b33f3fe671a4786a63bf18be61a4e52393bf92aded2563e5a93ff6245f248c18b33f36fbe8e36514b13f1e0d360549c3963f616f5285443f87bf95b234255fbdb6bf9916ad04046dc4bfc060cb858f4f79bf5704fec34870b1bfd945cea3bb9fa03facfaea645c7b95bfa82fae039313b03fd44909c73abc9e3feaf817c75efc9ebfee3d6a4542a7973f950fa0c55892a8bf15b135e5b6c1a63f52f1f1e503f879bfbcaac8e5b74379bfa39cd2a5286f493f22ad36277d829fbfb81c2c04a239a23f4912cce5965249bf531ecc635296a03fee892fe49948923fe666c983748ac03f9795e5838f05913fc3d41d6440fba1bf3943c8c37a85a0bfc385a1a6f7f66c3ff398bfc6567aad3f95c58b447cdb93bf372c47264f6cbb3fa0bc2b8677f47a3f88169de698e39c3f87571866bf9faa3fbcc675e5a4d987bf98702707ef3f9f3f8d4edde61cfcad3f8d12f8c60671bebfeb5ea065b393983f0605ef653feb793f87b7ab0543c5983f0ecfadc6a22c8dbf8d43d1e4080ba5bf512258852958a73f495b15e43bd6a1bf6749f364a69f953f383b04e46d8bb1bf84ffb7a4ab9cb4bf52739365435ba83f9618cc633996703f43e444e51b04a7bf100998c40d11a43f25981b447bf1713f28c78ba5bf39683f3cb87ac4ff90a33f62a94be471c3b2bf97d70c06846d9abf95c0d1e6a3c99dbf49cbb583ce34b0bf8b1c8324a8b593bfd641d1867ac7adbffe5dae049a72a43fa2c606c5c7f4953f90ff51c41fdfb23fa100094445a0a13fb2fb38a7908c7f3fb1be122789e57ebf28a429878c499f3ffdcfa886cf169dbfb9a425259d7bb6bffef75ae44e06a3bf16b83764556cb2bf1b655f25e177973f2e5857a6f3b2bbbfd364f606b269de3fd5fcd225cd7059bf751570a6031f9c3f4b0ab7e5b8f698bf86ba5cc4fe0d83bf49c445e4b1a9a23f778932e5eeb396bf7106ece52ade79bf6f89b7a620578dbf70d3fce3146b713f0aa53da43786a2bf4f2038841c6e923fcd258085f206883f90b919a450e9a13fecb6446692618bbf3c5a85c471bfa3bf9a7de904dc74a53fd747f2e57df9993f405c4ac63b7a9b3f084c52e46de0a2bfb4a7bae446a8943f71ef9d05108998bf6b6e1785883da63f4c16aea6d92d8dbff6dd12c59829663f0f2d8c46b9997cbfae262dc42b3ec2bf9a8f2306c1d08a3f45f542446d9db23f742d50c42bd792bf29d16fc6d81d8cbf5b8c19e5c746763f809438a554cea6bf1dab3944d974a2bfa2958bc4aadaa3bf15af54a51749973f5f17c6e43ada843fd99f6fc4886083bfdb8920041407a23fd2ce7024b465b33fe7e3bd8468b6a43f94cb1e46efbb8a3fabb03ac6c835abbf91ff3dc778a2afbfde126d86dd116cbf0ab97aa53fef87bff8ceec445a8375bfdc650a259a04a6bf07ec6c85f7b2973f2fb4e3c5d1b9893f5a289b24b11e84bf8475b7e4509a84bf8f7a16069c977abfb33e9824f81184bf291d3485d2bab6bf7cf6c3e6678d8d3f4832b8a34c3f90bf66b3cfe6adc0ad3ff461c7c3a28190bf7e0a3ac5b6d4863f74e506654ef555bfa3427e26f05c9c3f912a2de6b5fa9abfeaee2f658fa896bfe959d24642ccbd3fd3667ee551ffa73f90afbc03e952b0bf57c0ddc4944185bfbcf90c65dc0f96bfe448cc458553993f76f4a765d4b488bf27fa10455721963f54b8bc430f53a03fe057ac25ffc7983fdb5da78542b298bfa0a9d8645a2ba53f47310864bb9cb1bf2b2e8fc45fea73bf02f39e04412fa4bfff92c1c5bc2489bf963c0ae5e503a63f46e43c27a39d9fbf98e438a72b8c7f3fbcae8f6491eca3bffcfbc7a3438490bfb8d99944fb18943f93467365b8cea7bf75449724b30da4bf797e0985a700a6bfdb10126655849a3f816701c68d3baabfe1764825b713a7bf9b3d9ae4af1aa43f2e63db44413775bf20e03426631cbb3f853a75a67c357cbf0f25ea6630349ebf727072c64d298c3fa6a75ee6e1d26bbf5f49d303a2b5a03f752997c5796b983f7cb7ee04b08bb53f4b48c22498c984bff7828c05f43c983ff5ea5c64d20eb33f7800afa45f75943fda8325a78637af3ffaeaf8c6b8748ebf56efdfc3e1ec603feaacf866a9736e3f9b87ba84baa7a43ffe49f625000b7abfc5c227e755417f3f62603425f8bba63f472eb68634516dbf2249b4c3372ea03fe60424e58474963f1fa5a88614169d3fcd1765248832833fff5b7d84879c933f5075278703409fbf43ac2586fad9aabf3bc3dc8306df80bff96f86e42ec4933f933465c40533d3bf2dd19424ff02943f13db4886a9739bbf8182f686336a6ebf086e0b865867aabff75cd744ad25b5bfb23a9025314d983f112fb8c33e3fb0bf45bbe9e3ac17b1bfb43665240f33b33fc0465f255c7797bfd36fb0e5e0d9983f8d2b4ea71ce99f3ff92ddde3d8e0a0bf49a3a66513af983f2fedf203d73fa13fe8be0027ea968e3fc3b5aae6191f8d3f0e6db723ef3b90bf8f941d27de148fbf1676f38469a0753f5f74e4e39f00713fcb56d724922595bf48e25646f0b08bbf930b1185a321a63f09d1a6c

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Step 2 - Perform Vector Distance calculations</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The ClearScape Analytics function <a href = 'https://docs.teradata.com/r/Teradata-VantageCloud-Lake/Analyzing-Your-Data/Analytics-Database-Analytic-Functions/Model-Training-Functions/TD_VectorDistance'>TD_VectorDistance</a> function will take a table of input Vectors and a table of reference vectors and returns a table that contains the distance between target-reference pairs.  Since this function scans every row and performs the distance calculation, it is resource-intensive and usually suited to a lower number of records.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Additionally, perform a simple join to display the original complaints, topic of interest, and distance calculations</p>

In [26]:
SELECT TOP 10 target_id, reference_id, distancetype, cast(distance as decimal(36,8)) as distance FROM TD_VECTORDISTANCE (
    ON CFPB_Embeddings_Normalized AS TargetTable
    ON topics_embeddings_Normalized AS ReferenceTable DIMENSION
USING
    TargetIDColumn('id')
    TargetFeatureColumns('Vector_Data')
    RefIDColumn('id')
    RefFeatureColumns('Vector_Data')
    DistanceMeasure('cosine')
    topk(1)
) AS dt order by 3,1,2,4;

,target_id,reference_id,distancetype,distance
1,7720134,5,cosine,.34568930
2,7736083,2,cosine,.34685015
3,7756774,1,cosine,.36045400
4,7819325,5,cosine,.40321848
5,7838890,1,cosine,.41121418
6,7854870,5,cosine,.38240949
7,7907705,2,cosine,.34297853
8,7995584,3,cosine,.38756731
9,8279679,2,cosine,.37299986
10,8297421,2,cosine,.28438034


<hr>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Join the results back to the original complaints and topics</b></p>

In [27]:
SELECT TOP 2 * FROM demo_ofs.CFPB_Complaints;

,Date received,Product,Sub-product,Issue,Sub-issue,txt,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,id
1,2024-01-09,Credit card,General-purpose credit card or charge card,Incorrect information on your report,Account status incorrect,"It is entirely unacceptable to see late payments noted on my account, and I vehemently request that you promptly correct this mistake. Please take immediate action to rectify my account and guarantee its accurate reflection of my payment history.",Company has responded to the consumer and the CFPB and chooses not to provide a public response,Experian Information Solutions Inc.,TX,77042,None,Conse,Web,2024-01-09,Closed with explanation,Ye,**,8131638
2,2024-02-02,Debt collection,Other debt,Attempts to collect debt not owed,Debt was result of identity theft,"I'm disputing some accounts on my credit report due to identity theft. Other bureaus removed them, but yours hasn't. Creditors promised to fix it, so please remove the accounts ASAP.",Company has responded to the consumer and the CFPB and chooses not to provide a public response,Experian Information Solutions Inc.,CA,92201,None,Conse,Web,2024-02-02,Closed with non-monetary relief,Ye,**,8266390


In [28]:
SELECT TOP 2 * FROM demo_ofs.topics_of_interest;

,id,txt
1,2,Identity theft issues at Citibank
2,1,Fradulent activity with Debit Cards at Wells Fargo


In [29]:
SELECT TOP 10 c.id complaint_id, r.txt topic, c.txt complaint, d.distance

FROM (SELECT target_id, reference_id, distancetype, cast(distance as decimal(36,8)) as distance FROM TD_VECTORDISTANCE (
    ON CFPB_Embeddings_Normalized AS TargetTable
    ON topics_embeddings_Normalized AS ReferenceTable DIMENSION
USING
    TargetIDColumn('id')
    TargetFeatureColumns('Vector_Data')
    RefIDColumn('id')
    RefFeatureColumns('Vector_Data')
    DistanceMeasure('cosine')
    topk(1)
) AS dt) d
    
JOIN demo_ofs.CFPB_Complaints c
    ON c.id = d.target_id
JOIN demo_ofs.topics_of_interest r
    ON r.id = d.reference_id

ORDER BY d.distance;

,complaint_id,topic,complaint,distance
1,8297421,Identity theft issues at Citibank,A checking account was opened without my knowledge. I received a debt card in the mail XX/XX/XXXX telling me to activate the account. A few days letter I received another letter letting me know that the account was closed and to stop writing checks against the account. I did contact Citizens bank on XX/XX/XXXX around XXXX to let them know that I did not open the account. A representative put in a identity theft report for me.,.28438034
2,7907705,Identity theft issues at Citibank,I received a capital one card in the mail on XX/XX/XXXX I never requested nor do I even have an account with them after they made an inquiry on my experian account on XX/XX/XXXX. I never requested this inquiry either so they opened a credit card without my consent and brought my credit score down.,.34297853
3,7720134,App issues when transacting with bank,I have a credit card with goldman sachs/apple . I have had difficulties paying the card since having only XXXX payments. I requested them to work with my debt management company and they refused. I have called several times and also sent a letter to them asking if they would reduce interest and setup a payment plan with me directly. They have refused to work with me in any way and continue to only make the payment higher and more impossible for me to catch up on and have made the regular monthly payment besides the back payment unfairly high. I called today XX/XX/18 and asked if there was any way I could resume monthly payments as they were and they said that the regular monthly payment besides the back monthly payment was now more than $ XXXX which previously was $ XXXX. So even if I paid the back payments I would now have to pay a $ XXXX monthly payment which they know is impossible for me to pay and also seems to be abusive of the collections system. I have tried very hard to do everything in my power to w,.34568930
4,7736083,Identity theft issues at Citibank,"Hi! I received an email that I had a hard inquiry on my account, via XXXX XXXX. This morning I woke up to an email saying I had a Capital One credit card on the way ( I never signed up for ). I have XXXX in touch with Capital One and I am working on cancelling the account that was opened and the credit card.",.34685015
5,7756774,Fradulent activity with Debit Cards at Wells Fargo,"I received an email from FNBO-Direct on XX/XX/ confirming they received my application for an account. I was given an application number. I had not opened or requested an account and thought it was a fraudulent email from a fake bank. I confirmed that FNBO-Direct is the First National Bank of Omaha-Direct and I that are an actual bank. I forwarded the email to their report phishing attempts believing I was sent a fake email from the bank. On XX/XX/XXXX XXXX I was sent an automated email thanking me for reporting the phishing. On XX/XX/, I received an email from FNBO-Direct stating that Our attempts to make a trial deposit into your XXXX XXXX XXXX XXXX Checking account using the information provided in the application process have failed. I did not apply for an account with FNBO-Direct, nor do I have an account at the XXXX XXXX XXXX and therefore did not fill out an application with this information. On XX/XX/, at XXXX XXXX Eastern I called FNBO about this issue and spoke to someone who wanted me to gi",.36045400
6,8279679,Identity theft issues at Citibank,"Complaint Letter to Delete Inaccurate Information Dear Sir or Madam : I respectfully request your prompt attention to the following inaccurate information on my credit report. For your convenience, the disputed item is listed on the enclosed page of my credit report. The following item is inaccurate and negatively affects my credit. Please reinvestigate this matter and delete and/or correct the disputed item. I respectfully request that you provide me with proof of the disputed item, specifically any contract, note, court documents, o

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Step 3 - leverage an HSNW index for fast search</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The ClearScape Analytics function <a href = 'https://docs.teradata.com/r/Teradata-VantageCloud-Lake/Analyzing-Your-Data/Analytics-Database-Analytic-Functions/Model-Training-Functions/TD_VectorDistance'>TD_HSNW</a> (Hierarchical Navigable Small Worlds) function will create an index table representing....  This index model can then be used by the TD_HSNWPredict function to perform extremely efficient similarity searches.</p>

<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Train the model using the Vector Datatype as input</li>
    <li>Predict nearest matches using the topics embeddings as input</li>
    <li>Join the original data for human-readable results</li>
    </ol>
    
<hr>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Train the model</b></p>

In [35]:
SELECT * FROM TD_HNSW (
    ON CFPB_Embeddings_Normalized AS InputTable
    OUT VOLATILE TABLE OutputTable(hnsw_model)
USING
    IdColumn('id')
    VectorColumn('Vector_Data')
    EfConstruction(16)
    NumConnPerNode(16)
    MaxNumConnPerNode(20)
    DistanceMeasure('euclidean')
    EmbeddingSize(384)
    ApplyHeuristics('true')
) as dt;

ERROR: Unable to run SQL: Unable to run SQL query: Database reported error:3803:Table 'hnsw_model' already exists.

<hr>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Return similar results using topics as input</b></p>

In [36]:
SELECT TOP 10 * FROM
TD_HNSWPREDICT (
    ON hnsw_model AS ModelTable
    ON topics_embeddings_Normalized AS InputTable DIMENSION
    USING
    IdColumn('id')
    VectorColumn('Vector_Data')
    EfSearch(16)
    TopK(2)
    OutputNearestVector('true')
) T ORDER BY 1,3;

id nearest_neighbor_id distance nearest_neighbor_vector 1 1 8297421 0.8265277541568549 997689b62fefa23f00a615b6ead693bfc1786dd56a2795bfdd53099758ef81bf1de81b10bccb7fbfcd62bcb7128990bf2245d0974961c03f88bae1f75a3ea03f07c1f1b5bc1ea4bf38a8265039b69fbfd877fd948507a63fba6799722ed0aa3f95438cb279eaaabf35e37ef59104753ff96db5d154989cbfc86963d6513b533fce2a60b4424297bff721ba35078eb4bf1d39f1575a1f603fd5ad3d102988af3f873bb5d7629790bf1565e314b13b86bf3fefebb49a2aa6bf987f09363aef933f01bcf355c61aa43fd4f7b5949596b63fd0691e559ac575bf97cdabd197abacbfceb8a9f6a3aec2bf3f1fc9b2b470babf32c17df76306a13fe50369f139318d3f5ad85256785cb33f51da39b1978f9dbf47708fd701e3b03f34e939365c8e83bfd91dd511ee584cbff995d4938b59a83f6b8246b144768d3f740cdad7b84da0bfd0793ab1588e7d3f150113556edc753f62a516b6ebd4833f51a19636d7d462bfce71a8f216b2aa3ff820821388fe783f224c9c3747c9b03f6ffee333b73a88bf6ed203d695fab33f5b0e9236fedda23f85bd039786fa91bf2280d275455d843f811209f3bff0a93f44c8fd373906b03f4af955d08c579fbfeac64975d66ea53f52e254d72a58913f272edff41f4486bf1c65fd97ff06803fbab73c90158a9f3f149c1fd3a7c3a93fc188aed2e7a58abfacb88c528fe9babf33ebe5d63136a23f72fc21163bbe833fe4636930b330af3f5413b4910a9b8c3f3b53cbb0be6caebf449fa0f79fc0a03f44989dd03ec87e3fc5c524341ab9973f09df37d73792a1bffa7de9760b2f92bf87c93e769a84833f43e43055a1a065bf938851948a5f97bfeb735bb3ea4b993fc684fe36f90472bfaaaaf8142111863f79ce4875c770a53fbabba63411b5b6bf95699ff57dc394bf931d8cf1feeaac3f5105519203619bbf16596231913eadbfaf7e22d4a8bda73f412d29979eaf91bfdbf05014ba60973f961284f531fa943ff9706df7072791bf0a87c8b60071b23f810a8677cff5a03f6032d2b1c55eac3f85c8177417d3973f95217352c31cbbbf563d5b70034dafbf8f836317e53a913f80240636f1f5a33f979d0936feeeb3bfbfb61ed2abc5db3fb403b9b7d18f903fcea96254443d873f86962bd53dab953f38a04db7b066913fca17cb77a56ba03f4b045eb202479bbf3746c67267769abf02582d7681a7933f0ee26f1243238bbf94cbbc174188b0bf4ea5607032428fbf7094c6b4587596bf5a3993541adcb63fff3b4d942468a7bf3f744fb4b363973f30a83056e0a0b33fa185857612f7a23fb87d4b17f66a813f26d9a1d249bf6abf1b02d9b7cd4f903fd4b04e11e6657dbff78d8fd471e3a63f895d38b73a91a1bf100f3df54888a5bf38440af5e9edb5bfb2b4a43774b8b0bf87796e914d268dbfba328a177eedb03f71df4e166b64a3bf72b666729c358b3f19a831d38b9f893f4f96ecf0302a9ebf0c2392f50ddea4bfb4c38530edf79ebfa970eed7eb24903ff2b0869012f69ebf10c36eb0f325afbf29eed274a25ca63f86bba49766b8b03fc65c1a10d3ceafbf22322d5506a8853f5800e0d60842823fc07b04b227fa3bbf66954397c87a91bf5d6d68764931a3bfc783dc97c948803fa10f1cd7dcc9913fce6536b181969d3fa5c1b4b75698a03f5453ea77272da0bf7fc5f9b1960f5cbf264a351280988b3f54e9bb73ea8a383f3b40d5575257b0bf9658a01312c2983fbe308136bdff42bf707506116df6ad3fe9179e56e8c592bf36812b762fab93bf04b4d7376a52903fc5190f13b0e4a93f10e3a5116eb78cbffa2130b7b3a1b1bf95c58d333ce7a83f534dbcb73d89a03f1f2612945ddeb7bfbcda1bd67fcaa33f5efa1a5225cd9b3f941e81138d00993f144ec734e573663fce9b0a911fee8d3fffc3f0532921783fb78777954a1355bfcd395f51d0448dbfa4e02b7670aab33fa8e1cd758366b4bf48d7f5b23a17aa3fa2873a94918d87bfe3c58bb5c9ea94bfcee22c5317a9993f734a4b53416ca9bf7bd57950cc0f5f3f3f637e723d067b3f48b0a2900dbe8e3f2d5b6617353591bf152007d6f9f3933f59c363969e3ab3bf04140773bdf4a9bfa781af7483a3963f2aeab170969faebf456d941579d9a4bfc9ba1af016ceafbf02d40b3403eba7bf0fc4c7f5bf72a43f56c36917642eb1bfedb83953688fa9bf9b90cf545e63a6bfc39cbf179e82b0bf5be62d7182a79dbf5de9b0b3eca088bfb2b958b42651a73f62424616a775833f8214139480dca73f5f8473f06f1caf3ff5bfa311b5bb8cbf8284d114765fb63f8c9381f3a2ffa83f31947df7bd06b1bf18a20a5467eda73f927088b220f28abf7157f810ac129e3f435151b1a4609d3fa649f156721fa23f48a4c1b6c77e92bf631a89d2ccf08a3f40b68a35e9eca4bfdb9b48f46571873f79f5e61074358e3f81a31df7b4c6513f942cec91cb2a8cbfc398d61031568ebf47a14156ea7ea3bf05fa4054ab80a73f937dab5553abd4bf3fdd065746f4a1bf1a8304d3dff999bf5a500a9698eda33f62d0899260ef9abf6cdc4fb73762b1bfa7ac9b32a4cb8a3fc61c8db51be8a4bffc227d52be089bbf2e05d256025e72bfb47a6d9084289fbfeb1ef732ab147a3f7219e9779b2f80bf271bea54432e96bfffbc9536a0d6923f2047a3d688bbb23f60d20a95cdec95bfcf0b8e553de6a43ff9c5087292f18bbff93d479006758fbfef420ad35eeea9bf07c0aa95c

<hr>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Join the results back to the original complaints and topics</b></p>

In [37]:
SELECT TOP 10 c.id complaint_id, r.txt topic, c.txt complaint, d.distance

FROM (SELECT * FROM
TD_HNSWPREDICT (
    ON hnsw_model AS ModelTable
    ON topics_embeddings_Normalized AS InputTable DIMENSION
    USING
    IdColumn('id')
    VectorColumn('Vector_Data')
    EfSearch(16)
    TopK(2)
    OutputNearestVector('true')
) T) d
    
    
JOIN demo_ofs.CFPB_Complaints c
    ON c.id = d.nearest_neighbor_id
JOIN demo_ofs.topics_of_interest r
    ON r.id = d.id

ORDER BY d.distance;

,complaint_id,topic,complaint,distance
1,8297421,Identity theft issues at Citibank,A checking account was opened without my knowledge. I received a debt card in the mail XX/XX/XXXX telling me to activate the account. A few days letter I received another letter letting me know that the account was closed and to stop writing checks against the account. I did contact Citizens bank on XX/XX/XXXX around XXXX to let them know that I did not open the account. A representative put in a identity theft report for me.,0.7541622358478657
2,8297421,Fradulent activity with Debit Cards at Wells Fargo,A checking account was opened without my knowledge. I received a debt card in the mail XX/XX/XXXX telling me to activate the account. A few days letter I received another letter letting me know that the account was closed and to stop writing checks against the account. I did contact Citizens bank on XX/XX/XXXX around XXXX to let them know that I did not open the account. A representative put in a identity theft report for me.,0.8265277541568549
3,7907705,Identity theft issues at Citibank,I received a capital one card in the mail on XX/XX/XXXX I never requested nor do I even have an account with them after they made an inquiry on my experian account on XX/XX/XXXX. I never requested this inquiry either so they opened a credit card without my consent and brought my credit score down.,0.8282252455116907
4,7720134,App issues when transacting with bank,I have a credit card with goldman sachs/apple . I have had difficulties paying the card since having only XXXX payments. I requested them to work with my debt management company and they refused. I have called several times and also sent a letter to them asking if they would reduce interest and setup a payment plan with me directly. They have refused to work with me in any way and continue to only make the payment higher and more impossible for me to catch up on and have made the regular monthly payment besides the back payment unfairly high. I called today XX/XX/18 and asked if there was any way I could resume monthly payments as they were and they said that the regular monthly payment besides the back monthly payment was now more than $ XXXX which previously was $ XXXX. So even if I paid the back payments I would now have to pay a $ XXXX monthly payment which they know is impossible for me to pay and also seems to be abusive of the collections system. I have tried very hard to do everything in my power to w,0.8314917937167623
5,7720134,Fradulent activity with Debit Cards at Wells Fargo,I have a credit card with goldman sachs/apple . I have had difficulties paying the card since having only XXXX payments. I requested them to work with my debt management company and they refused. I have called several times and also sent a letter to them asking if they would reduce interest and setup a payment plan with me directly. They have refused to work with me in any way and continue to only make the payment higher and more impossible for me to catch up on and have made the regular monthly payment besides the back payment unfairly high. I called today XX/XX/18 and asked if there was any way I could resume monthly payments as they were and they said that the regular monthly payment besides the back monthly payment was now more than $ XXXX which previously was $ XXXX. So even if I paid the back payments I would now have to pay a $ XXXX monthly payment which they know is impossible for me to pay and also seems to be abusive of the collections system. I have tried very hard to do everything in my power to w,0.8372770119410285
6,7756774,App issues when transacting with bank,"I received an email from FNBO-Direct on XX/XX/ confirming they received my application for an account. I was given an application number. I had not opened or requested an account and thought it was a fraudulent email from a fake bank. I confirmed that FNBO-Direct is the First National Bank of Omaha-Direct and I that are an actual bank. I forwarded 

<hr>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Use TD_HNSWSummary to create a human-readable model output</b></p>

In [39]:
SELECT amp_id, graph_id, node_id, layer_id, input_row_id, cast(node_vector
as varchar(60)) as node_vector, num_neighbors, cast(neighbor_node_id as
varchar(60)) as neighbor_node_id, cast(model_info as varchar(500)) as model_info 

FROM TD_HNSWSummary(
    ON hnsw_model as ModelTable
) as dt
ORDER by 1,9

,amp_id,graph_id,node_id,layer_id,input_row_id,node_vector,num_neighbors,neighbor_node_id,model_info
1,0,0,1,3,7819325,"-0.0543087,0.0116584,0.0230143,-0.0352023,-0.0153569,-0.0317",0,,
2,0,0,1,5,7819325,"-0.0543087,0.0116584,0.0230143,-0.0352023,-0.0153569,-0.0317",0,,
3,0,0,1,6,7819325,"-0.0543087,0.0116584,0.0230143,-0.0352023,-0.0153569,-0.0317",0,,
4,0,0,1,7,7819325,"-0.0543087,0.0116584,0.0230143,-0.0352023,-0.0153569,-0.0317",0,,
5,0,0,1,4,7819325,"-0.0543087,0.0116584,0.0230143,-0.0352023,-0.0153569,-0.0317",0,,
6,0,0,1,8,7819325,"-0.0543087,0.0116584,0.0230143,-0.0352023,-0.0153569,-0.0317",0,,
7,0,0,1,2,7819325,"-0.0543087,0.0116584,0.0230143,-0.0352023,-0.0153569,-0.0317",0,,
8,0,0,1,1,7819325,"-0.0543087,0.0116584,0.0230143,-0.0352023,-0.0153569,-0.0317",0,,
9,0,0,,,,,,,"NumLayer: 8, NumDataPoints: 1, EntryPointNodeId: 1, EfConstruction: 16, Seed: 849765878, ApplyHeuristics: true, NumConnPerNode: 16, MaxNumConnPerNode: 20, DistanceMeasure: EUCLIDEAN, EmbeddingSize: 384, NodeCountInLayer: [1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1]"
10,1,0,1,3,8279679,"-0.0488655,-0.0410246,0.00882475,0.0110491,0.038276,-0.07640",0,,


In [69]:
%disconnect vs_demo

Success: 'vs_demo' disconnected